<a href="https://colab.research.google.com/github/obengdouglas/DSI-2022/blob/main/Machine_Translation_ACLED1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.1 MB/s 
     |████████████████████████████████| 895 kB 44.4 MB/s 
     |████████████████████████████████| 6.6 MB 38.3 MB/s 
     |████████████████████████████████| 77 kB 7.0 MB/s 
     |████████████████████████████████| 596 kB 42.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
#Retrieve pipeline of modules 
from transformers import pipeline
import pandas as pd
import csv
from tqdm import tqdm
import time
from datetime import timedelta

In [3]:
#Choose English to French translation
translator = pipeline("translation_en_to_fr")

No model was supplied, defaulted to t5-base (https://huggingface.co/t5-base)


Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [4]:
tqdm.pandas()

In [5]:

#read csv file from github repo
acled = pd.read_csv('https://raw.githubusercontent.com/obengdouglas/DSI-2022/main/module3-nlp-twist/Data/ACLED-2022-01-01-2022-03-31-Western_Africa.csv', sep=",", nrows = 3005, skip_blank_lines=True, na_filter=True)
acled.tail()

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
3000,8785781,566,NIG23115,23115,01-Jan-22,2022,1,Violence against civilians,Abduction/forced disappearance,Unidentified Armed Group (Nigeria),...,Shendam,8.8833,9.5333,1,Nigeria Punch; Sun (Nigeria); Premium Times (N...,National-Regional,"On 1 January 2022, unidentified armed men abdu...",0,1642439768,NGA
3001,8785992,566,NIG23261,23261,01-Jan-22,2022,1,Explosions/Remote violence,Shelling/artillery/missile attack,Islamic State (West Africa) - Lake Chad Faction,...,Damasak,13.0977,12.4952,1,Twitter,New media,"On 1 January 2022, ISWAP claimed a mortar atta...",0,1642439769,NGA
3002,8793203,566,NIG23277,23277,01-Jan-22,2022,1,Battles,Armed clash,Fulani Ethnic Militia (Nigeria),...,Abraka,5.7902,6.1047,2,Nigeria Punch,National,"On 1 January 2022, hunters clashed with armed ...",0,1643040848,NGA
3003,8843285,854,BFO6142,6142,01-Jan-22,2022,1,Battles,Armed clash,Islamic State (West Africa) - Greater Sahara F...,...,Tokabangou,14.6402,0.1525,1,Sahel Security Alerts; Facebook; Twitter,New media,"On 1 January 2022, presumed ISWAP (Greater Sah...",0,1645032563,BFA
3004,8849517,854,BFO6144,6144,01-Jan-22,2022,1,Violence against civilians,Abduction/forced disappearance,JNIM: Group for Support of Islam and Muslims,...,Wousse,13.5406,-1.5690,1,MinuteBf; Sahel Security Alerts; Facebook; Twi...,New media,"On 1 January 2022, presumed JNIM militants abd...",0,1645470967,BFA


In [6]:
acled.describe()

,data_id,iso,event_id_no_cnty,year,time_precision,inter1,inter2,interaction,latitude,longitude,geo_precision,fatalities,timestamp
count,3.005000e+03,3005.000000,3005.000000,3005.0,3005.000000,3005.000000,3005.000000,3005.000000,3005.000000,3005.000000,3005.000000,3005.000000,3.005000e+03
mean,8.862195e+06,608.848586,13146.991015,2022.0,1.104493,3.294176,4.063894,33.983694,11.397137,1.318363,1.416306,2.172379,1.645760e+09
std,7.086547e+04,172.981084,9080.343374,0.0,0.323919,1.820686,2.989091,17.524229,3.292023,6.951856,0.510933,9.140760,2.248261e+06
min,8.704653e+06,132.000000,116.000000,2022.0,1.000000,1.000000,0.000000,10.000000,4.315100,-25.065000,1.000000,0.000000,1.641864e+09
25%,8.801516e+06,562.000000,6264.000000,2022.0,1.000000,2.000000,1.000000,17.000000,9.183300,-2.322100,1.000000,0.000000,1.643646e+09
50%,8.849952e+06,566.000000,6816.000000,2022.0,1.000000,3.000000,4.000000,27.000000,11.917900,0.984700,1.000000,0.000000,1.645471e+09
75%,8.909971e+06,854.000000,23586.000000,2022.0,1.000000,5.000000,7.000000,47.000000,13.803100,6.885400,2.000000,1.000000,1.647889e+09
max,9.107243e+06,854.000000,24503.000000,2022.0,3.000000,8.000000,8.000000,78.000000,25.861000,14.299300,3.000000,225.000000,1.650317e+09


In [7]:
start_time = time.monotonic()


In [8]:
acled["notes_fr"] = acled.notes.progress_apply(translator)
acled.head()

100%|██████████| 3005/3005 [8:07:36<00:00,  9.74s/it]


,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3,notes_fr
0,8974302,854,BFO6949,6949,31-Mar-22,2022,1,Violence against civilians,Attack,JNIM: Group for Support of Islam and Muslims a...,...,13.6451,-0.7156,2,Facebook; L'Observateur Paalga; Infowakat; Min...,New media-National,"On 31 March 2022, presumed JNIM or IS Sahel mi...",20,1649088162,BFA,"[{'translation_text': 'Le 31 mars 2022, des mi..."
1,8974306,854,BFO6964,6964,31-Mar-22,2022,1,Strategic developments,Looting/property destruction,JNIM: Group for Support of Islam and Muslims,...,11.9333,1.6167,1,Gulmu Info; Facebook; Undisclosed Source,Local partner-Other,"Looting: On 31 March 2022, presumed JNIM milit...",0,1649088162,BFA,[{'translation_text': 'Le pillage : Le 31 mars...
2,8974377,566,NIG24258,24258,31-Mar-22,2022,2,Battles,Armed clash,Boko Haram - Jamaatu Ahli is-Sunnah lid-Dawati...,...,13.6169,13.2644,2,Daily Post (Nigeria); Sahara Reporters; Daily ...,National-Regional,"Around 31 March 2022 (as reported), Boko Haram...",20,1649088162,NGA,[{'translation_text': 'Vers le 31 mars 2022 (t...
3,8974378,566,NIG24259,24259,31-Mar-22,2022,1,Battles,Armed clash,Unidentified Armed Group (Nigeria),...,6.2658,6.5973,2,Nigeria Punch; Guardian (Nigeria),National,"On 31 March 2022, armed individuals attacked s...",1,1649088162,NGA,"[{'translation_text': 'Le 31 mars 2022, des in..."
4,8974400,566,NIG24300,24300,31-Mar-22,2022,1,Strategic developments,Disrupted weapons use,Police Forces of Nigeria (2015-),...,10.5422,7.3302,1,Sahara Reporters; Daily Trust (Nigeria); Niger...,National-Regional,"Defusal: On 31 March 2022, operatives of the a...",0,1649088162,NGA,[{'translation_text': 'Défaut : Le 31 mars 202...


In [9]:
end_time = time.monotonic()
print(timedelta(seconds=end_time - start_time))

8:07:36.885580


In [10]:
#Output saved in github repo https://github.com/obengdouglas/DSI-2022/tree/main/module3-nlp-twist/Data
acled.to_csv("acled_fr.csv", index = False)

In [ ]:
while True: pass